## 載入需要的模組

In [69]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation

## 讀入資料

In [70]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv("gender_submission.csv")

## 先看一下資料內容

In [71]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 看看缺失值的狀況

In [72]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [73]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


## 看看資料的分布狀況

In [74]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [75]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


## 從上面觀察的結果，選取自己想要的特徵

In [76]:
selected_features = ['Pclass','Age','Fare','Sex']

In [77]:
X_train = train[selected_features]

In [78]:
y_train = train['Survived']

In [79]:
X_test = test[selected_features]

## 處理缺失值

In [80]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
Pclass    891 non-null int64
Age       714 non-null float64
Fare      891 non-null float64
Sex       891 non-null object
dtypes: float64(2), int64(1), object(1)
memory usage: 27.9+ KB


In [81]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
Pclass    418 non-null int64
Age       332 non-null float64
Fare      417 non-null float64
Sex       418 non-null object
dtypes: float64(2), int64(1), object(1)
memory usage: 13.1+ KB


In [82]:
X_train['Age'].fillna(X_train['Age'].mean(), inplace=True)
X_test['Age'].fillna(X_test['Age'].median(), inplace=True)
X_test['Fare'].fillna(X_test['Fare'].median(), inplace=True)

C:\Users\damy\Anaconda3\lib\site-packages\pandas\core\generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [83]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
Pclass    891 non-null int64
Age       891 non-null float64
Fare      891 non-null float64
Sex       891 non-null object
dtypes: float64(2), int64(1), object(1)
memory usage: 27.9+ KB


## 將字串類型的類別資料做轉換

In [84]:
X_train = pd.get_dummies(X_train)

In [85]:
X_train.head()

,Pclass,Age,Fare,Sex_female,Sex_male
0,3,22.0,7.2500,0,1
1,1,38.0,71.2833,1,0
2,3,26.0,7.9250,1,0
3,1,35.0,53.1000,1,0
4,3,35.0,8.0500,0,1


In [86]:
X_test = pd.get_dummies(X_test)

In [87]:
X_test.head()

,Pclass,Age,Fare,Sex_female,Sex_male
0,3,34.5,7.8292,0,1
1,3,47.0,7.0000,1,0
2,2,62.0,9.6875,0,1
3,3,27.0,8.6625,0,1
4,3,22.0,12.2875,1,0


## 使用 Random Forest 來做分類

In [88]:
rfc = RandomForestClassifier(criterion='gini', 
                             n_estimators=1000,
                             min_samples_split=15,
                             min_samples_leaf=1,
                             random_state=1,
                             ) 

## 使用 K-Fold Cross Validation 看一下分類的準確度

In [89]:
cross_val_score(rfc, X_train, y_train, cv=10).mean()

0.84301781863579617

## 訓練模型

In [90]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=15,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

## 用訓練好的模型來做預測

In [91]:
survived_predict = rfc.predict(X_test)

## Keras tenserflower

In [150]:
model = Sequential()
model.add(Dense(32, input_dim=5,activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1,activation='sigmoid'))


In [155]:
# 多分类问题
model.compile(optimizer='adam',
              loss='poisson',
              metrics=['accuracy'])

In [156]:
model.fit(X_train,y_train, epochs=32, batch_size=32)

Epoch 1/32
891/891 [==============================] - 1s 601us/step - loss: 0.6455 - acc: 0.7789
Epoch 2/32
891/891 [==============================] - 0s 76us/step - loss: 0.6399 - acc: 0.7811
Epoch 3/32
891/891 [==============================] - 0s 63us/step - loss: 0.6481 - acc: 0.7744
Epoch 4/32
891/891 [==============================] - 0s 76us/step - loss: 0.6410 - acc: 0.7845
Epoch 5/32
891/891 [==============================] - 0s 76us/step - loss: 0.6410 - acc: 0.7823
Epoch 6/32
891/891 [==============================] - 0s 76us/step - loss: 0.6429 - acc: 0.7789
Epoch 7/32
891/891 [==============================] - 0s 73us/step - loss: 0.6372 - acc: 0.7823
Epoch 8/32
891/891 [==============================] - 0s 72us/step - loss: 0.6476 - acc: 0.7800
Epoch 9/32
891/891 [==============================] - 0s 99us/step - loss: 0.6379 - acc: 0.7767
Epoch 10/32
891/891 [==============================] - 0s 76us/step - loss: 0.6435 - acc: 0.7811
Epoch 11/32
891/891 [=================